<a href="https://colab.research.google.com/github/lettymoon/analise-temperatura-sp/blob/main/temperaturas_bairros_sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1) Apresentação

#2) Bibliotecas

<a fazer>

In [68]:
import os
import io
import re
import numpy as np
import pandas as pd
import requests
import time
from tqdm import tqdm
from datetime import datetime

#3) Pipeline 1 - Bairros(Geodados)

In [88]:
# @title Extração de dados

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

query = """
[out:json][timeout:240];

area
  ["boundary"="administrative"]
  ["admin_level"="8"]
  ["name"="São Paulo"]
->.cityArea;

(
  node(area.cityArea)["place"="suburb"]["name"];
  way(area.cityArea)["place"="suburb"]["name"];
  relation(area.cityArea)["place"="suburb"]["name"];
);

out tags center;
"""

headers = {
    "User-Agent": "colab-overpass-neighborhoods/1.0 (contact: you@example.com)"
}

resp = requests.post(
    OVERPASS_URL,
    data={"data": query},
    headers=headers,
    timeout=300
)
resp.raise_for_status()
data = resp.json()

rows = []
for el in data.get("elements", []):
    name = el.get("tags", {}).get("name")
    if not name:
        continue

    if el["type"] == "node":
        lat, lon = el.get("lat"), el.get("lon")
    else:
        center = el.get("center", {})
        lat, lon = center.get("lat"), center.get("lon")

    if lat is not None and lon is not None:
        rows.append({
            "bairro": name.strip(),
            "lat": float(lat),
            "lon": float(lon),
            "osm_type": el["type"],
            "osm_id": el["id"],
        })

bairros = pd.DataFrame(rows).drop_duplicates(subset=["bairro"]).reset_index(drop=True)

In [71]:
# @title Armazenamento de dados - Bronze (Parquet + CSV)
os.makedirs("data/bairros/bronze/parquet", exist_ok=True)
os.makedirs("data/bairros/bronze/csv", exist_ok=True)

os.makedirs("data/bairros/silver/parquet", exist_ok=True)
os.makedirs("data/bairros/silver/csv", exist_ok=True)

base_name = "bairros_capital_sp"

bronze_parquet_path = f"data/bairros/bronze/parquet/bairros_capital_sp.parquet"
bronze_csv_path = f"data/bairros/bronze/csv/bairros_capital_sp.csv"

bairros.to_parquet(bronze_parquet_path, index=False)
bairros.to_csv(bronze_csv_path, index=False)

bronze_path = os.path.join("data/bairros/bronze/parquet", "bairros_capital_sp.parquet")

print(bronze_parquet_path)
print(bronze_csv_path)

data/bairros/bronze/parquet/bairros_capital_sp.parquet
data/bairros/bronze/csv/bairros_capital_sp.csv


In [72]:
bairros

,bairro,lat,lon,osm_type,osm_id
0,Interlagos,-23.701167,-46.708291,node,205197916
1,Tatuapé,-23.540891,-46.575009,node,243072564
2,Marsilac,-23.908391,-46.706105,node,368179749
3,Parelheiros,-23.827312,-46.727794,node,368179752
4,São Mateus,-23.598299,-46.481705,node,368179763
...,...,...,...,...,...
314,Alto do Rivieira,-23.699364,-46.761721,way,253716659
315,Jardim Helga,-23.642960,-46.773885,way,253717214
316,Jardim Rosana,-23.646518,-46.774449,way,253717845
317,Jardim das Oliveiras,-23.493248,-46.378116,way,1384106767


In [73]:
# @title Auditoria

import numpy as np

audit = {}

audit["linhas_total"] = len(bairros)
audit["colunas"] = list(bairros.columns)

# Nulos por coluna
audit["nulos_por_coluna"] = bairros.isna().sum().to_dict()

# Duplicados por bairro (antes de remover)
audit["duplicados_bairro"] = int(bairros.duplicated(subset=["bairro"]).sum())

# Duplicados exatos (linha inteira)
audit["duplicados_linha_inteira"] = int(bairros.duplicated().sum())

# Checar lat/lon fora de faixa
audit["lat_fora_faixa"] = int(((bairros["lat"] < -90) | (bairros["lat"] > 90)).sum())
audit["lon_fora_faixa"] = int(((bairros["lon"] < -180) | (bairros["lon"] > 180)).sum())

# Valores únicos
audit["bairros_unicos"] = int(bairros["bairro"].nunique())

# Resumo numérico de lat/lon
lat_desc = bairros["lat"].describe().to_dict()
lon_desc = bairros["lon"].describe().to_dict()

audit_df = pd.DataFrame([
    {"metrica": "linhas_total", "valor": audit["linhas_total"]},
    {"metrica": "bairros_unicos", "valor": audit["bairros_unicos"]},
    {"metrica": "duplicados_bairro", "valor": audit["duplicados_bairro"]},
    {"metrica": "duplicados_linha_inteira", "valor": audit["duplicados_linha_inteira"]},
    {"metrica": "lat_fora_faixa", "valor": audit["lat_fora_faixa"]},
    {"metrica": "lon_fora_faixa", "valor": audit["lon_fora_faixa"]},
])

print("Auditoria (resumo):")
display(audit_df)

print("Nulos por coluna:")
display(pd.DataFrame([audit["nulos_por_coluna"]]).T.rename(columns={0:"nulos"}))

# Visualização rápida: 10 bairros mais repetidos (se houver)
top_rep = (bairros["bairro"].value_counts().head(10).reset_index())
top_rep.columns = ["bairro", "contagem"]
print("10 bairros mais repetidos (se >1, vale tratar):")
display(top_rep)


Auditoria (resumo):


,metrica,valor
0,linhas_total,319
1,bairros_unicos,319
2,duplicados_bairro,0
3,duplicados_linha_inteira,0
4,lat_fora_faixa,0
5,lon_fora_faixa,0


Nulos por coluna:


,nulos
bairro,0
lat,0
lon,0
osm_type,0
osm_id,0


10 bairros mais repetidos (se >1, vale tratar):


,bairro,contagem
0,Jardim Amaralina,1
1,Interlagos,1
2,Tatuapé,1
3,Marsilac,1
4,Parelheiros,1
5,São Mateus,1
6,Sapopemba,1
7,Santa Margarida Paulista,1
8,Vila Leopoldina,1
9,Casa Verde,1


In [74]:
# @title Tratamento dos dados - Silver

bairros_silver = bairros.copy()

# Limpeza do nome do bairro
bairros_silver["bairro"] = bairros_silver["bairro"].astype(str).str.strip()

# Garantir tipos numéricos
bairros_silver["lat"] = pd.to_numeric(bairros_silver["lat"], errors="coerce")
bairros_silver["lon"] = pd.to_numeric(bairros_silver["lon"], errors="coerce")

# Remover bairros com lat e lon nulos
bairros_silver = bairros_silver.dropna(subset=["bairro", "lat", "lon"])

# Remover bairros repetidos, menos o último
bairros_silver = bairros_silver.drop_duplicates(subset=["bairro"]).reset_index(drop=True)

bairros_silver["location_id"] = bairros_silver.index.astype(int)

bairros_silver = bairros_silver[["location_id", "bairro", "lat", "lon"]]

display(bairros_silver.head(10))
print("\nTotal final (silver):", len(bairros_silver))


,location_id,bairro,lat,lon
0,0,Interlagos,-23.701167,-46.708291
1,1,Tatuapé,-23.540891,-46.575009
2,2,Marsilac,-23.908391,-46.706105
3,3,Parelheiros,-23.827312,-46.727794
4,4,São Mateus,-23.598299,-46.481705
5,5,Sapopemba,-23.604326,-46.509885
6,6,Santa Margarida Paulista,-23.518102,-46.398910
7,7,Vila Leopoldina,-23.527503,-46.732947
8,8,Casa Verde,-23.505927,-46.656138
9,9,Limão,-23.503315,-46.675272



Total final (silver): 319


In [75]:
# @title Armazenamento de dados - Silver (Parquet + CSV)

silver_parquet_path = f"data/bairros/silver/parquet/bairros_capital_sp.parquet"
silver_csv_path = f"data/bairros/silver/csv/bairros_capital_sp.csv"

bairros_silver.to_parquet(silver_parquet_path, index=False)
bairros_silver.to_csv(silver_csv_path, index=False)

print(silver_parquet_path)
print(silver_csv_path)

data/bairros/silver/parquet/bairros_capital_sp.parquet
data/bairros/silver/csv/bairros_capital_sp.csv


In [76]:
bairros_silver

,location_id,bairro,lat,lon
0,0,Interlagos,-23.701167,-46.708291
1,1,Tatuapé,-23.540891,-46.575009
2,2,Marsilac,-23.908391,-46.706105
3,3,Parelheiros,-23.827312,-46.727794
4,4,São Mateus,-23.598299,-46.481705
...,...,...,...,...
314,314,Alto do Rivieira,-23.699364,-46.761721
315,315,Jardim Helga,-23.642960,-46.773885
316,316,Jardim Rosana,-23.646518,-46.774449
317,317,Jardim das Oliveiras,-23.493248,-46.378116


#4) Pipeline 2 - Temperatura(Clima)

In [81]:
# @title Extração de dados

!pip install requests

# API OpenWeatherMap
OPEN_METEO_URL = "https://api.open-meteo.com/v1/forecast"
session = requests.Session()

bairros_silver = pd.read_parquet("data/bairros/silver/parquet/bairros_capital_sp.parquet")

daily_vars = [ "temperature_2m_max",
              "temperature_2m_min",
               "apparent_temperature_max",
               "apparent_temperature_min",
               "weather_code",
               "dew_point_2m_max",
               "dew_point_2m_min",
               "temperature_2m_mean",
               "apparent_temperature_mean",
               "relative_humidity_2m_mean",
               "relative_humidity_2m_max",
              "relative_humidity_2m_min",
               "precipitation_probability_mean",
               "precipitation_probability_min", ]

def fetch_daily_weather_batch(batch_df: pd.DataFrame, timezone: str = "America/Sao_Paulo") -> pd.DataFrame:
    params = {
        "latitude": ",".join(batch_df["lat"].astype(str).tolist()),
        "longitude": ",".join(batch_df["lon"].astype(str).tolist()),
        "daily": ",".join(daily_vars),
        "timezone": timezone,
        "format": "csv",
    }

    r = session.get(OPEN_METEO_URL, params=params, timeout=60)
    r.raise_for_status()

    text = r.text.strip()
    blocks = re.split(r"\n\s*\n", text)
    daily_block = None
    for b in blocks:
        if b.lstrip().startswith("location_id,time"):
            daily_block = b
            break

    if daily_block is None:
        raise ValueError("Não encontrei o bloco daily no CSV. Primeiro 300 chars:\n" + text[:300])

    df = pd.read_csv(io.StringIO(daily_block))

    df.columns = [re.sub(r"\s*\(.*?\)\s*$", "", c).strip() for c in df.columns]

    if "time" in df.columns:
        df = df.rename(columns={"time": "weather_time"})

    return df


# Coletar em lotes

CHUNK_SIZE = 100
SLEEP_SECONDS = 0.2

weather_chunks = []
errors = 0

for start in tqdm(range(0, len(bairros_silver), CHUNK_SIZE)):
    batch = bairros_silver.iloc[start:start + CHUNK_SIZE]
    try:
        w = fetch_daily_weather_batch(batch)
        w["location_id"] = w["location_id"].astype(int) + start
        weather_chunks.append(w)

    except Exception as e:
        errors += 1
        fallback = pd.DataFrame({
            "location_id": batch["location_id"].values,
            "weather_time": [None] * len(batch),
            **{k: [None] * len(batch) for k in daily_vars},
        })

        weather_chunks.append(fallback)

    time.sleep(SLEEP_SECONDS)

clima_df = pd.concat(weather_chunks, ignore_index=True)

# Salvar em Parquet
os.makedirs("data/clima/bronze/parquet", exist_ok=True)
os.makedirs("data/clima/bronze/csv", exist_ok=True)
stamp = datetime.now().strftime("%Y%m%dT%")
out_parquet = f"data/clima/bronze/parquet/clima_sp_{stamp}.parquet"
out_csv = f"data/clima/bronze/csv/clima_sp_{stamp}.csv"

clima_df.to_parquet(out_parquet, index=False)
clima_df.to_csv(out_csv, index=False)

print(f"Clima salvo em:\n{out_parquet}\n{out_csv}")

100%|██████████| 4/4 [00:02<00:00,  1.50it/s]

Clima salvo em:
data/clima/bronze/parquet/clima_sp_20260108T%.parquet
data/clima/bronze/csv/clima_sp_20260108T%.csv


In [82]:
weather_df[(weather_df['weather_time'] ==  '2026-01-12')]

,location_id,weather_time,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,weather_code,dew_point_2m_max,dew_point_2m_min,temperature_2m_mean,apparent_temperature_mean,relative_humidity_2m_mean,relative_humidity_2m_max,relative_humidity_2m_min,precipitation_probability_mean,precipitation_probability_min
4,0,2026-01-12,30.5,19.2,32.6,21.9,80,20.7,14.9,23.6,26.2,77,98,39,37,15
11,1,2026-01-12,32.5,22.6,34.9,26.9,3,21.3,14.6,26.6,29.5,67,92,34,36,9
18,2,2026-01-12,29.4,18.6,32.8,21.0,80,20.3,16.6,22.7,25.6,80,98,46,50,9
25,3,2026-01-12,29.5,18.7,32.9,21.2,80,20.4,16.7,22.8,25.7,80,98,46,37,15
32,4,2026-01-12,31.1,21.8,33.2,24.7,80,20.4,12.4,25.0,27.1,66,87,33,36,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,314,2026-01-12,30.3,19.1,32.4,21.7,80,20.5,14.8,23.4,26.0,77,98,39,37,15
2209,315,2026-01-12,30.4,18.7,31.9,21.0,3,19.5,14.9,23.5,25.6,74,95,39,37,15
2216,316,2026-01-12,30.2,18.4,31.6,20.7,3,19.3,14.7,23.3,25.3,74,95,39,37,15
2223,317,2026-01-12,33.1,22.0,34.7,25.2,3,21.1,11.0,26.5,28.8,62,85,26,36,9


In [83]:
# @title Auditoria

# Auditoria dos dados de clima
audit_weather = {}

# Contagem de nulos por coluna
audit_weather["nulos_por_coluna"] = weather_df.isna().sum().to_dict()

# Verificar se temos valores fora da faixa de temperatura
audit_weather["temp_max_fora_faixa"] = len(weather_df[weather_df["temperature_2m_max"] < -100])
audit_weather["temp_min_fora_faixa"] = len(weather_df[weather_df["temperature_2m_min"] < -100])

# Resumo numérico de temperaturas
audit_weather["temp_max_desc"] = weather_df["temperature_2m_max"].describe().to_dict()
audit_weather["temp_min_desc"] = weather_df["temperature_2m_min"].describe().to_dict()

# Exemplo de visualização de clima
audit_df = pd.DataFrame([
    {"metrica": "temp_max_fora_faixa", "valor": audit_weather["temp_max_fora_faixa"]},
    {"metrica": "temp_min_fora_faixa", "valor": audit_weather["temp_min_fora_faixa"]},
])

print("Auditoria de clima (resumo):")
display(audit_df)

# Visualizando os nulos por coluna
print("Nulos por coluna de clima:")
display(pd.DataFrame([audit_weather["nulos_por_coluna"]]).T.rename(columns={0:"nulos"}))

# Exibindo descrição das temperaturas
print("Descrição das temperaturas máximas:")
display(pd.DataFrame([audit_weather["temp_max_desc"]]))
print("Descrição das temperaturas mínimas:")
display(pd.DataFrame([audit_weather["temp_min_desc"]]))

Auditoria de clima (resumo):


,metrica,valor
0,temp_max_fora_faixa,0
1,temp_min_fora_faixa,0


Nulos por coluna de clima:


,nulos
location_id,0
weather_time,0
temperature_2m_max,0
temperature_2m_min,0
apparent_temperature_max,0
apparent_temperature_min,0
weather_code,0
dew_point_2m_max,0
dew_point_2m_min,0
temperature_2m_mean,0


Descrição das temperaturas máximas:


,count,mean,std,min,25%,50%,75%,max
0,2233.0,30.284102,1.961443,23.5,28.8,30.7,31.8,33.6


Descrição das temperaturas mínimas:


,count,mean,std,min,25%,50%,75%,max
0,2233.0,20.526825,1.299577,16.9,19.6,20.8,21.6,22.8


In [84]:
# @title Tratamento dos dados - Silver

bairros_silver = pd.read_parquet("data/bairros/silver/parquet/bairros_capital_sp.parquet")

# Merge: clima (bronze) com os bairros (silver)
bairros_weather_silver = bairros_silver.merge(
    weather_df[["location_id", "weather_time"] + daily_vars],
    on="location_id", how="left"
)

In [85]:
# @title Armazenamento dos dados

os.makedirs("data/clima/silver/parquet", exist_ok=True)
os.makedirs("data/clima/silver/csv", exist_ok=True)
out_silver_parquet = f"data/clima/silver/parquet/bairros_clima_sp_{stamp}.parquet"
out_silver_csv = f"data/clima/silver/csv/bairros_clima_sp_{stamp}.csv"

bairros_weather_silver.to_parquet(out_silver_parquet, index=False)
bairros_weather_silver.to_csv(out_silver_csv, index=False)

print(f"{out_silver_parquet}\n{out_silver_csv}")

data/clima/silver/parquet/bairros_clima_sp_20260108T%.parquet
data/clima/silver/csv/bairros_clima_sp_20260108T%.csv


In [86]:
#@title Checkpoint

# dataset final para análise
bairros_weather_final = pd.read_parquet(out_silver_parquet)

print(f"Dados carregados para análise com {len(bairros_weather_final)} bairros.")

Dados carregados para análise com 2233 bairros.


In [87]:
bairros_weather_final

,location_id,bairro,lat,lon,weather_time,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,weather_code,dew_point_2m_max,dew_point_2m_min,temperature_2m_mean,apparent_temperature_mean,relative_humidity_2m_mean,relative_humidity_2m_max,relative_humidity_2m_min,precipitation_probability_mean,precipitation_probability_min
0,0,Interlagos,-23.701167,-46.708291,2026-01-08,29.2,17.4,32.4,19.4,95,19.9,15.7,22.4,24.7,78,99,44,17,0
1,0,Interlagos,-23.701167,-46.708291,2026-01-09,30.0,19.1,32.5,21.8,81,21.0,13.9,23.3,26.1,78,98,38,9,0
2,0,Interlagos,-23.701167,-46.708291,2026-01-10,31.4,19.1,33.5,21.5,80,19.0,14.6,24.2,26.5,71,96,37,9,0
3,0,Interlagos,-23.701167,-46.708291,2026-01-11,31.3,18.1,33.4,20.0,3,20.0,16.0,24.5,26.7,70,94,41,15,0
4,0,Interlagos,-23.701167,-46.708291,2026-01-12,30.5,19.2,32.6,21.9,80,20.7,14.9,23.6,26.2,77,98,39,37,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,318,Jardim Amaralina,-23.604204,-46.798635,2026-01-10,30.6,18.9,32.5,21.3,3,19.2,15.0,24.3,26.4,69,96,39,4,0
2229,318,Jardim Amaralina,-23.604204,-46.798635,2026-01-11,30.3,18.5,31.6,20.5,3,19.2,15.6,24.2,26.0,69,90,41,9,0
2230,318,Jardim Amaralina,-23.604204,-46.798635,2026-01-12,30.2,18.5,31.6,20.7,3,19.3,14.7,23.3,25.4,74,95,39,33,14
2231,318,Jardim Amaralina,-23.604204,-46.798635,2026-01-13,27.2,17.8,30.6,20.0,95,19.4,17.6,22.4,25.2,81,99,62,48,23


#5) Análise (Gold)